![](../pictures/AirflowLogo.png)

# Apache Airflow Deployment

To start the deployment, we need to be logged into the cluster:

In [2]:
# Replace the command with your own one inside the single quotes and run the cell
# Example OC_LOGIN_COMMAND='oc login --token=sha256~3bR5KXgwiUoaQiph2_kIXCDQnVfm_HQy3YwU2m-UOrs --server=https://c109-e.us-east.containers.cloud.ibm.com:31656'
OC_LOGIN_COMMAND='_paste_here_your_login_command_'
$OC_LOGIN_COMMAND

Logged into "https://c102-e.us-east.containers.cloud.ibm.com:31723" as "IAM#angelito@de.ibm.com" using the token provided.

You have access to 68 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "databand".


In order to identify and separate Airflow from the rest of the cluster, we create a project called `airflow`

In [26]:
oc new-project airflow 

Already on project "airflow" on server "https://c102-e.us-east.containers.cloud.ibm.com:31723".

You can add applications to this project with the 'new-app' command. For example, try:

    oc new-app rails-postgresql-example

to build a new example application in Ruby. Or use kubectl to deploy a simple Kubernetes application:

    kubectl create deployment hello-node --image=k8s.gcr.io/e2e-test-images/agnhost:2.33 -- /agnhost serve-hostname



Now, we get the official helm charts for Airflow:

In [27]:
helm repo add apache-airflow https://airflow.apache.org
helm repo update

"apache-airflow" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "apache-airflow" chart repository
Update Complete. ⎈Happy Helming!⎈


And the actual deployment starts now:

In [28]:
helm upgrade --install airflow apache-airflow/airflow --namespace airflow

Release "airflow" does not exist. Installing it now.
NAME: airflow
LAST DEPLOYED: Thu Mar  2 16:42:30 2023
NAMESPACE: airflow
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Thank you for installing Apache Airflow 2.5.1!

Your release is named airflow.
You can now access your dashboard(s) by executing the following command(s) and visiting the corresponding port at localhost in your browser:

Airflow Webserver:     kubectl port-forward svc/airflow-webserver 8080:8080 --namespace airflow
Default Webserver (Airflow UI) Login credentials:
    username: admin
    password: admin
Default Postgres connection credentials:
    username: postgres
    password: postgres
    port: 5432

You can get Fernet Key value by running the following:

    echo Fernet Key: $(kubectl get secret --namespace airflow airflow-fernet-key -o jsonpath="{.data.fernet-key}" | base64 --decode)

###########################################################
#  WARNING: You should set a static webserver secret key  #
#

After five minutes you may get a timeout error. and if you take a look at the OpenShift console, you may see errors like this:

![](../pictures/airflowerror.png)

In that case, you need to change these security settings:

In [20]:
oc adm policy add-scc-to-group anyuid system:serviceaccounts:airflow   
oc adm policy add-scc-to-group privileged system:serviceaccounts:airflow 

clusterrole.rbac.authorization.k8s.io/system:openshift:scc:anyuid added: "system:serviceaccounts:airflow"
clusterrole.rbac.authorization.k8s.io/system:openshift:scc:privileged added: "system:serviceaccounts:airflow"


And re-try the deployment:

In [22]:
helm upgrade airflow apache-airflow/airflow --namespace airflow 

Release "airflow" has been upgraded. Happy Helming!
NAME: airflow
LAST DEPLOYED: Thu Mar  2 16:36:12 2023
NAMESPACE: airflow
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
Thank you for installing Apache Airflow 2.5.1!

Your release is named airflow.
You can now access your dashboard(s) by executing the following command(s) and visiting the corresponding port at localhost in your browser:

Airflow Webserver:     kubectl port-forward svc/airflow-webserver 8080:8080 --namespace airflow
Default Webserver (Airflow UI) Login credentials:
    username: admin
    password: admin
Default Postgres connection credentials:
    username: postgres
    password: postgres
    port: 5432

You can get Fernet Key value by running the following:

    echo Fernet Key: $(kubectl get secret --namespace airflow airflow-fernet-key -o jsonpath="{.data.fernet-key}" | base64 --decode)

###########################################################
#  WARNING: You should set a static webserver secret key  #
##

If the deployment went well, you will see several pods running:

In [30]:
oc get pods

NAME                                 READY   STATUS    RESTARTS      AGE
airflow-postgresql-0                 1/1     Running   0             4m2s
airflow-redis-0                      1/1     Running   0             4m2s
airflow-scheduler-6c7b454c-zrbzg     2/2     Running   0             4m2s
airflow-statsd-d8c8f886c-mdffw       1/1     Running   0             4m2s
airflow-triggerer-559ffb9496-f8jmh   1/1     Running   0             4m2s
airflow-webserver-758c974499-vxldl   0/1     Running   1 (54s ago)   4m2s
airflow-worker-0                     2/2     Running   0             4m2s


Next, you we need to add a route to the custer to access Airflow:

In [31]:
oc get svc
oc expose svc airflow-webserver 

NAME                    TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
airflow-postgresql      ClusterIP   172.21.171.109   <none>        5432/TCP            4m13s
airflow-postgresql-hl   ClusterIP   None             <none>        5432/TCP            4m13s
airflow-redis           ClusterIP   172.21.75.90     <none>        6379/TCP            4m13s
airflow-statsd          ClusterIP   172.21.67.35     <none>        9125/UDP,9102/TCP   4m13s
airflow-webserver       ClusterIP   172.21.123.197   <none>        8080/TCP            4m13s
airflow-worker          ClusterIP   None             <none>        8793/TCP            4m13s
route.route.openshift.io/airflow-webserver exposed


And, just like we did with databand, we get access url from the OpenShift console:

![](../pictures/airflowroute.png)

You can login with the default user: `admin` and default password: `admin`. Then you will see the main dashboard of Airflow

![](../pictures/airflowempty.png)

Indeed, the main dashboard is empty because the default helm deployment disables the default DAGs. If you really miss them, you can modify an environment and re-deploy Airflow

In [ ]:
# helm upgrade airflow apache-airflow/airflow -f - << EOF
# extraEnv: |
#   - name: AIRFLOW__CORE__LOAD_EXAMPLES
#     value: 'True'" > vs-override.yaml
# EOF

In [32]:
# We need to generate a small file. Choose a temporary directory and paste it in the next line
cd '_any_temporary_directory_with_write_permission_'
echo "extraEnv: |
  - name: AIRFLOW__CORE__LOAD_EXAMPLES
    value: 'True'" > vs-override.yaml

helm upgrade airflow apache-airflow/airflow -f vs-override.yaml

Release "airflow" has been upgraded. Happy Helming!
NAME: airflow
LAST DEPLOYED: Thu Mar  2 16:49:20 2023
NAMESPACE: airflow
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
Thank you for installing Apache Airflow 2.5.1!

Your release is named airflow.
You can now access your dashboard(s) by executing the following command(s) and visiting the corresponding port at localhost in your browser:

Airflow Webserver:     kubectl port-forward svc/airflow-webserver 8080:8080 --namespace airflow
Default Webserver (Airflow UI) Login credentials:
    username: admin
    password: admin
Default Postgres connection credentials:
    username: postgres
    password: postgres
    port: 5432

You can get Fernet Key value by running the following:

    echo Fernet Key: $(kubectl get secret --namespace airflow airflow-fernet-key -o jsonpath="{.data.fernet-key}" | base64 --decode)

###########################################################
#  WARNING: You should set a static webserver secret key  #
##

And now, you will see them:

![](../pictures/airflowdefault.png)


---

Next Section: [Airflow integration](./4_airflow_int.ipynb).   Previous Section: [Databand deployment](./2_databand_deploy.ipynb)

[Return to main](../README.md)
